In [ ]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M  # https://docs.python.org/2/library/math.html

In [ ]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6                                          # [m] zonal extent 
L_y = 2e6                                          # [m] meridional extent 
A = L_x*L_y                                        # [m^2] surface area of interior 
P = 2*L_x+2*L_y                                    # [m] Perimeter 

g = 9.81                                           # [m/s^2] gravitational acceleration 
C_p = 3.994e3                                      # [J/(kg K)] specific heat capacity of sea water 
s = 0.02                                           # [] average bottom slope around the perimeter of the marginal sea
f_0 = 1.2e-4                                       # [s^-1] Coriolis parameter 
alpha_T = 0.2                                      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
rho_ref = 1028                                     # [kg m^-3] reference density sea water
T_ref = 8                                          # [Celcius] reference temperature sea water
S_ref = 35                                         # [ppt] reference salinity sea water

#### Parameters ####
num_val = 100
A_T_1 = np.append(8.87, np.linspace(7,11,num_val))       # [Celsius] ingoing temperature 
A_T_A = np.append(2.8, np.linspace(0,7,num_val))         # [Celsius] Atmospheric temperature 
A_E = np.append(-2e-8,np.linspace(0,-5e-8,num_val))      # [m/s] net evaporation minus precipitation
A_Gamma = np.append(10,np.linspace(10,60,num_val))       # [W/(m^2 C)] Restoring strength 
A_H = np.append(1000, np.linspace(600,1200,num_val))     # [m] sill height 
A_c = np.append(0.006, np.linspace(0.003,0.009,num_val)) # [] eddy coefficient 

#### Preparation #####
T_0 = np.zeros(num_val)
T_2 = np.zeros(num_val)
A_D_rho = np.zeros(num_val)
A_W = np.zeros(num_val)
Delta_T = np.zeros(num_val)
M_D_rho = np.zeros((num_val, num_val))

In [ ]:
############# Computation ##################
############################################

#### Parameterization ####
def func_epsilon(c):
    return c*P/L

def func_mu(Gamma, H, T_1, T_A):
    return A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)) #[-]

#### Two main functions #####
def Temp_0(T_1, mu, epsilon, T_A):
    return T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)

def Temp_2(T_1, T_0, mu, T_A, L):
    return T_1 - 2*mu*((T_1-T_A)/(T_1-T_0)*(1+P*L/A)-1)*(T_1-T_A)

#### Follow up computations ####
def downwelling(H, T_1, T_2):
    return (-1)*g*H**2*(-alpha_T*(T_1-T_2))/(4*rho_ref*f_0)

def Delta_density(T, T_ref):
    return (-alpha_T*(T-T_ref))

#def Q(T_0, T_A, L, Gamma, T_1) :
#    return A*(T_0 - T_A)*Gamma + P*L*Gamma*(T_1-T_A)
#def Delta(T_0, T_1, T_2):
#    return (T_2-T_0)/(T_1-T_0)

In [ ]:
################ Output ####################
############################################

y=7

if y==1:          ######## Delta T ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[1::][i], A_T_A[0], A_E[0], A_Gamma[0], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        
    var1_T_0, var1_T_2 = T_0, T_2
    %store var1_T_0
    %store var1_T_2

if y==2:          ######## Varying Eddy Coefficient c ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[1::][i], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[0], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_D_rho[i]=Delta_density(T_1,T_2[i])

    var2_T_0, var2_T_2, var2_rho = T_0, T_2, A_D_rho
    %store var2_T_0
    %store var2_T_2
    %store var2_rho
    
elif y==3:
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[0],  A_T_A[0], A_E[0], A_Gamma[1::][i], A_H[0] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_D_rho[i]=Delta_density(T_1,T_2[i])
        
    var3_T_0, var3_T_2, var3_rho = T_0, T_2, A_D_rho
    %store var3_T_0
    %store var3_T_2
    %store var3_rho
    
if y==4:          ######## Delta T ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[0], A_H[1::][i] 
        L = H/s  # [m] width of sloping topography of the boundary current 
        
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_W[i]=downwelling(H, T_1, T_2[i])
    var4_W = A_W
    %store var4_W

if y==5:          ######## Downwelling ##########
    for i in range(num_val):
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[1::][i], A_T_A[0], A_E[0], A_Gamma[0], A_H[0]
        L = H/s  # [m] width of sloping topography of the boundary current 
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_W[i]=downwelling(H, T_1, T_2[i])*1e-6
        
    var5_W = A_W
    %store var5_W
        
for j in range(num_val):
        
        c, T_1, T_A, E, Gamma, H  = A_c[0], A_T_1[0], A_T_A[0], A_E[0], A_Gamma[1::][j], A_H[0]
        L = H/s  # [m] width of sloping topography of the boundary current 
        epsilon = func_epsilon(c)
        mu = func_mu(Gamma, H, T_1, T_A)
        T_0[j]=Temp_0(T_1, mu, epsilon, T_A)
        T_2[j]=Temp_2(T_1, T_0[i], mu, T_A, L)
        A_W[j]=downwelling(H, T_1, T_2[j])*1e-6     

    var6_W = A_W
    %store var6_W
    
if y==6:          ######## Contour: evaporation and restoring strength for density difference ##########
    for i in range(num_val):
        E =  A_E[1::][i]
        for j in range(num_val):
            c, T_1, T_A, Gamma, H  = A_c[0], A_T_1[0], A_T_A[0], A_Gamma[1::][j], A_H[0]
            L = H/s  # [m] width of sloping topography of the boundary current 
            epsilon = func_epsilon(c)
            mu = func_mu(Gamma, H, T_1, T_A)
            T_0[i]=Temp_0(T_1, mu, epsilon, T_A)
            T_2[i]=Temp_2(T_1, T_0[i], mu, T_A, L)
            M_D_rho[i,j]=Delta_density(T_1, T_2[i])

    x = A_Gamma[1::]
    y = A_E[1::]*1e8
    z = M_D_rho
    xlabel = r'Restoring Strength $\Gamma$ [$\frac{W}{m^2 K}$]'
    ylabel =  r'Meridional Overturning Strength W [$\frac{m^2}{s} \times 10^{6} $]'
    label = ['Salt Model','Simple Model']
    
    fig = plt.figure()
    cp = plt.contourf(x,y,z[:,:])
    cbar = plt.colorbar(cp)
    cbar.set_label(r'Density Difference [$\frac{kg}{m^3}$]')
    plt.xlabel(r'Restoring Strength $\Gamma$ [$\frac{W}{m^2 K}$]')
    plt.ylabel(r'Net Evaporation minus Precipitation E [$\frac{m}{s} \times 10^{-8}$]')
    #plt.show()
    plt.savefig('Contour-simple.png', dpi=300, bbox_inches='tight') 